# start

In [ ]:
# constansts

START_WORD_LENGTHS = [4,5,6]

MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 9

NUM_STEPS = 8
MIN_STEPS = 5


In [ ]:
'''
Created on Feb 7, 2013

@author: olegs
'''

ROMAN_CONSTANTS = (
            ( "", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX" ),
            ( "", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC" ),
            ( "", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM" ),
            ( "", "M", "MM", "MMM", "",   "",  "-",  "",    "",     ""   ),
        )

ROMAN_SYMBOL_MAP = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)

CUTOFF = 4000
BIG_DEC = 2900
BIG_ROMAN = "MMCM"
ROMAN_NOUGHT = "nulla"

def digits(num):
    if num < 0:
        raise Exception('range error: negative numbers not supported')
    if num % 1 != 0.0:
        raise Exception('floating point numbers not supported')
    res = []
    while num > 0:
        res.append(num % 10)
        num //= 10
    return res

def toString(num, emptyZero=False):
    if num < CUTOFF:
        digitlist = digits(num)
        if digitlist:
            res = reversed([ ROMAN_CONSTANTS[order][digit] for order, digit in enumerate(digitlist) ])
            return "".join(res)
        else:
            return "" if emptyZero else ROMAN_NOUGHT
    else:
        if num % 1 != 0.0:
            raise Exception('floating point numbers not supported')
        # For numbers over or equal the CUTOFF, the remainder of division by 2900
        # is represented as above, prepended with the multiples of MMCM (2900 in Roman),
        # which guarantees no more than 3 repetitive Ms.
        return BIG_ROMAN * (num // BIG_DEC) + toString(num % BIG_DEC, emptyZero=True)

def parse(numeral):
    numeral = numeral.upper()
    result = 0

    lastVal = 0
    lastCount = 0
    subtraction = False
    for symbol in numeral[::-1]:
        value = ROMAN_SYMBOL_MAP.get(symbol)
        if not value:
            return False
        if lastVal == 0:
            lastCount = 1
            lastVal = value
        elif lastVal == value:
            lastCount += 1
            # exceptions
        else:
            result += (-1 if subtraction else 1) * lastVal * lastCount
            subtraction = lastVal > value
            lastCount = 1
            lastVal = value

    result = result + (-1 if subtraction else 1) * lastVal * lastCount
    return True

# loadDict

In [ ]:
import requests
import time


LOAD_FROM_URL = True

def normalizeWord(word):
    return word.strip().upper()

def loadDict(dictionaryFileName):
  t0 = time.time()

  lines = []
  if LOAD_FROM_URL:
      baseUrl = 'https://azrafe7.github.io/'
      f = requests.get(baseUrl + dictionaryFileName)
      lines = f.text.splitlines()
  else:
      with open(dictionaryFileName) as f:
          lines = f.read().splitlines()

  filteredWords = []
  for line in lines:
      word = normalizeWord(line)
      length = len(word)
      if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH and not parse(word):
          filteredWords.append(word)

  print("Dictionary '{0}' loaded in {1:.2f}s".format(dictionaryFileName, (time.time() - t0)))

  print('{0}/{1}'.format(len(filteredWords), len(lines)))

  print(filteredWords[:10])

  return filteredWords

In [ ]:
import requests
import time

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'
wordsDictionaryFileName = 'assets/Ita_scr.txt'
validationDictionaryFileName = 'assets/Ita.txt'

filteredWords = loadDict(wordsDictionaryFileName)
validationDatabase = set(loadDict(validationDictionaryFileName))

In [ ]:

def wordsOfLength(lengths):
    return [word for word in filteredWords if len(word) in lengths]

len(list(wordsOfLength(START_WORD_LENGTHS)))


# load into DataFrame

In [ ]:
def countOccurrences(word):
  counts = [0 for j in range(ord('Z') - ord('A') + 1)]
  for ch in word:
    counts[ord(ch) - ord('A')] += 1
  return counts

In [ ]:
import pandas as pd
import numpy as np
import time

t0 = time.time()

TOTAL_WORDS = len(filteredWords)

NUM_WORDS = TOTAL_WORDS

raw_data = [countOccurrences(w) for w in filteredWords[:NUM_WORDS]]
print(raw_data[:10])

data = np.array(raw_data, dtype=np.int8)
df = pd.DataFrame(data=data, index=[w for w in filteredWords[:NUM_WORDS]], columns=[chr(ord('A') + i) for i in range(data.shape[1])])
# df.insert(loc=0, column='WORD', value=[w for w in filteredWords[:NUM_WORDS]])

pd.options.display.max_columns = None

print(f'Elapsed {(time.time() - t0):.2f}s')

display(df.head(10))

In [ ]:

def isAnagram(sourceWord, targetWord):
    if len(sourceWord) != len(targetWord):
        return False

    res = True

    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)

    #print(targetCharSet)
    for i, ch in enumerate(sourceWord):
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
        else:
            res = False
            break

    return res


assert(isAnagram('', ''))
assert(not isAnagram('corto', 'trOca'))
assert(not isAnagram('c', ''))
assert(isAnagram('cortao', 'trocao'))

srcWord = 'ostra'
dstWord ='motri'

print('isAnagram("{0}", "{1}")'.format(srcWord, dstWord))
print(isAnagram(srcWord, dstWord))

In [ ]:
import time

def isAnagramEx(sourceWord, targetWord, maxDiffLength=0, strict=False):
    if maxDiffLength == 0:
        return isAnagram(sourceWord, targetWord)
    else:
        wordDiffLength = abs(len(sourceWord) - len(targetWord))
        if wordDiffLength > maxDiffLength:
            return False

    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)

    if len(sourceCharSet) > len(targetCharSet):
        sourceCharSet, targetCharSet = targetCharSet, sourceCharSet

    #print(targetCharSet, sourceCharSet)
    for ch in sourceCharSet:
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)

    #print(targetCharSet)
    targetCharSetLength = len(targetCharSet)
    #print(wordDiffLength, targetCharSet)
    isLooseAnagram = (targetCharSetLength <= maxDiffLength)
    res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and wordDiffLength == maxDiffLength)
    return res


maxDiffLength=2

assert(isAnagramEx('', ''))
assert(not isAnagramEx('corto', 'trOca'))
assert(not isAnagramEx('c', ''))
assert(isAnagramEx('cortao', 'trocao'))
assert(isAnagramEx('cortao', 'trocaox', 1))
assert(not isAnagramEx('strappo', 'posta', 1))
assert(isAnagramEx('strappo', 'posta', 2))
assert(isAnagramEx('strappo', 'posta', 3))
assert(not isAnagramEx('strappo', 'posta', 3, strict=True))
assert(isAnagramEx('strappo', 'posta', 2, strict=True))
assert(isAnagramEx('LEVI', 'ABILE', 2,))
assert(not isAnagramEx('TUFFAI', 'ABACA', 2,))


srcWord = 'straccio'
dstWord ='crosta'

print('isAnagramEx("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(0):
  isAnagramEx('', '')
  not isAnagramEx('corto', 'trOca')
  not isAnagramEx('c', '')
  isAnagramEx('cortao', 'trocao')
  isAnagramEx('cortao', 'trocaox', 1)
  not isAnagramEx('strappo', 'posta', 1)
  isAnagramEx('strappo', 'posta', 2)
  isAnagramEx('strappo', 'posta', 3)
  not isAnagramEx('strappo', 'posta', 3, strict=True)
  isAnagramEx('strappo', 'posta', 2, strict=True)
  isAnagramEx('LEVI', 'ABILE', 2, )
  not isAnagramEx('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


# cy_isAnagramEx2

In [ ]:
# %load_ext cython

In [ ]:
# %%cython
# import cython
# import time

# @cython.boundscheck(False)
# def _cy_isAnagramEx2(list sortedSourceChars, list sortedTargetChars, int sourceLength, int targetLength, int maxDiffLength=0, bint strict=False, bint failFast=False):
#   if failFast and abs(sourceLength - targetLength) > maxDiffLength:
#     return False, -1

#   # if maxDiffLength == 0:
#   #     return sortedSourceChars == sortedTargetChars
#   # else:
#   #   wordDiffLength = abs(sourceLength - targetLength)
#   #   if wordDiffLength > maxDiffLength:
#   #       return False

#   cdef int i = 0
#   cdef int j = 0
#   cdef int diff = 0
#   cdef int same = 0
#   if sourceLength != 0 and targetLength != 0:
#     while i < sourceLength and j < targetLength:
#       sourceCh = sortedSourceChars[i]
#       targetCh = sortedTargetChars[j]
#       if sourceCh == targetCh:
#         i += 1
#         j += 1
#         same += 1
#       else:
#         if sourceCh < targetCh:
#           i += 1
#         else:
#           j += 1

#   diff = max(sourceLength, targetLength) - same
#   #print(sortedSourceChars, sortedTargetChars, diff, same)
#   cdef bint isLooseAnagram = diff <= maxDiffLength
#   cdef bint res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)
#   return res, diff


# cpdef cy_isAnagramEx2(str sourceWord, str targetWord, int maxDiffLength=0, bint strict=False, bint failFast=False):
#   cdef list sortedSourceChars = sorted(sourceWord)
#   cdef list sortedTargetChars = sorted(targetWord)

#   # if maxDiffLength == 0:
#   #     return sortedSourceChars == sortedTargetChars
#   # else:


#   cdef int sourceLength = len(sortedSourceChars)
#   cdef int targetLength = len(sortedTargetChars)

#   return _cy_isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=maxDiffLength, strict=strict, failFast=failFast)

# maxDiffLength = 2

# assert (True, 0) == cy_isAnagramEx2('', '')
# assert (False, 2) == cy_isAnagramEx2('corto', 'trOca')
# assert (False, 4) == cy_isAnagramEx2('corto', 'trOcaul')
# assert (False, 1) == cy_isAnagramEx2('c', '')
# assert (True, 1) == cy_isAnagramEx2('cortao', 'trocaox', 1)
# assert (True, 0) == cy_isAnagramEx2('cortao', 'trocao')
# assert (False, 2) == cy_isAnagramEx2('strappo', 'posta', 1)
# assert (True, 2) == cy_isAnagramEx2('strappo', 'posta', 2)
# assert (True, 2) == cy_isAnagramEx2('strappo', 'posta', 3)
# assert (False, 2) == cy_isAnagramEx2('strappo', 'posta', 3, strict=True)
# assert (True, 2) == cy_isAnagramEx2('strappo', 'posta', 2, strict=True)
# assert (True, 2) == cy_isAnagramEx2('LEVI', 'ABILE', 2, strict=False)
# assert (False, 5) == cy_isAnagramEx2('TUFFAI', 'ABACA', 2, strict=False)
# assert (False, 5) == cy_isAnagramEx2('TUFFAI', 'ABACA', 2, strict=True)
# assert (True, 1) == cy_isAnagramEx2('AOPST', 'AOPSTU', 1, strict=False)
# assert (True, 1) == cy_isAnagramEx2('AOPST', 'AOPSTU', 1, strict=True)

# srcWord = 'straccio'
# dstWord = 'crosta'

# print('cy_isAnagramEx2("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
# print(cy_isAnagramEx2(srcWord, dstWord, maxDiffLength=maxDiffLength))

# t0 = time.time()
# for i in range(0):
#   cy_isAnagramEx2('', '')
#   not cy_isAnagramEx2('corto', 'trOca')
#   not cy_isAnagramEx2('c', '')
#   cy_isAnagramEx2('cortao', 'trocao')
#   cy_isAnagramEx2('cortao', 'trocaox', 1)
#   not cy_isAnagramEx2('strappo', 'posta', 1)
#   cy_isAnagramEx2('strappo', 'posta', 2)
#   cy_isAnagramEx2('strappo', 'posta', 3)
#   not cy_isAnagramEx2('strappo', 'posta', 3, strict=True)
#   cy_isAnagramEx2('strappo', 'posta', 2, strict=True)
#   cy_isAnagramEx2('LEVI', 'ABILE', 2, )
#   not cy_isAnagramEx2('TUFFAI', 'ABACA', 2, )
# print(f'{(time.time() - t0):.4f}s')


In [ ]:

# t0 = time.time()
# for i in range(0):
#   cy_isAnagramEx2('', '')
#   not cy_isAnagramEx2('corto', 'trOca')
#   not cy_isAnagramEx2('c', '')
#   cy_isAnagramEx2('cortao', 'trocao')
#   cy_isAnagramEx2('cortao', 'trocaox', 1)
#   not cy_isAnagramEx2('strappo', 'posta', 1)
#   cy_isAnagramEx2('strappo', 'posta', 2)
#   cy_isAnagramEx2('strappo', 'posta', 3)
#   not cy_isAnagramEx2('strappo', 'posta', 3, strict=True)
#   cy_isAnagramEx2('strappo', 'posta', 2, strict=True)
#   cy_isAnagramEx2('LEVI', 'ABILE', 2, )
#   not cy_isAnagramEx2('TUFFAI', 'ABACA', 2, )
# print(f'{(time.time() - t0):.4f}s')

In [ ]:
# %%cython
# import cython
# from cpython cimport array
# import array
# import time
# from libc.math cimport abs

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef _cy_isAnagramEx3(str sortedSourceChars, str sortedTargetChars, int sourceLength, int targetLength, int maxDiffLength=0, bint strict=False, bint failFast=False):
#   if failFast and abs(sourceLength - targetLength) > maxDiffLength:
#     return False, -1

#   # if maxDiffLength == 0:
#   #     return sortedSourceChars == sortedTargetChars
#   # else:
#   #   wordDiffLength = abs(sourceLength - targetLength)
#   #   if wordDiffLength > maxDiffLength:
#   #       return False

#   cdef Py_ssize_t i = 0
#   cdef Py_ssize_t j = 0
#   cdef int diff = 0
#   cdef int same = 0
#   if sourceLength != 0 and targetLength != 0:
#     while i < sourceLength and j < targetLength:
#       sourceCh = sortedSourceChars[i]
#       targetCh = sortedTargetChars[j]
#       if sourceCh == targetCh:
#         i += 1
#         j += 1
#         same += 1
#       else:
#         if sourceCh < targetCh:
#           i += 1
#         else:
#           j += 1

#   diff = max(sourceLength, targetLength) - same
#   #print(sortedSourceChars, sortedTargetChars, diff, same)
#   cdef bint isLooseAnagram = diff <= maxDiffLength
#   cdef bint res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)
#   return res, diff

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cpdef cy_isAnagramEx3(str sourceWord, str targetWord, int maxDiffLength=0, bint strict=False, bint failFast=False):
#   cdef str sortedSourceChars = ''.join(sorted(sourceWord))
#   cdef str sortedTargetChars = ''.join(sorted(targetWord))

#   cdef Py_ssize_t sourceLength = len(sortedSourceChars)
#   cdef Py_ssize_t targetLength = len(sortedTargetChars)

#   # cdef int[:] sourceArray = array.array('u', sortedSourceChars)
#   # cdef int[:] targetArray = array.array('u', sortedTargetChars)
#   # if maxDiffLength == 0:
#   #     return sortedSourceChars == sortedTargetChars
#   # else:


#   if failFast and abs(sourceLength - targetLength) > maxDiffLength:
#     return False, -1

#   # if maxDiffLength == 0:
#   #     return sortedSourceChars == sortedTargetChars
#   # else:
#   #   wordDiffLength = abs(sourceLength - targetLength)
#   #   if wordDiffLength > maxDiffLength:
#   #       return False

#   cdef Py_ssize_t i = 0
#   cdef Py_ssize_t j = 0
#   cdef int diff = 0
#   cdef int same = 0
#   if sourceLength != 0 and targetLength != 0:
#     while i < sourceLength and j < targetLength:
#       sourceCh = sortedSourceChars[i]
#       targetCh = sortedTargetChars[j]
#       if sourceCh == targetCh:
#         i += 1
#         j += 1
#         same += 1
#       else:
#         if sourceCh < targetCh:
#           i += 1
#         else:
#           j += 1

#   diff = max(sourceLength, targetLength) - same
#   #print(sortedSourceChars, sortedTargetChars, diff, same)
#   cdef bint isLooseAnagram = diff <= maxDiffLength
#   cdef bint res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)


#   return res, diff
#   return _cy_isAnagramEx3(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=maxDiffLength, strict=strict, failFast=failFast)


# maxDiffLength = 2

# assert (True, 0) == cy_isAnagramEx3('', '')
# assert (False, 2) == cy_isAnagramEx3('corto', 'trOca')
# assert (False, 4) == cy_isAnagramEx3('corto', 'trOcaul')
# assert (False, 1) == cy_isAnagramEx3('c', '')
# assert (True, 1) == cy_isAnagramEx3('cortao', 'trocaox', 1)
# assert (True, 0) == cy_isAnagramEx3('cortao', 'trocao')
# assert (False, 2) == cy_isAnagramEx3('strappo', 'posta', 1)
# assert (True, 2) == cy_isAnagramEx3('strappo', 'posta', 2)
# assert (True, 2) == cy_isAnagramEx3('strappo', 'posta', 3)
# assert (False, 2) == cy_isAnagramEx3('strappo', 'posta', 3, strict=True)
# assert (True, 2) == cy_isAnagramEx3('strappo', 'posta', 2, strict=True)
# assert (True, 2) == cy_isAnagramEx3('LEVI', 'ABILE', 2, strict=False)
# assert (False, 5) == cy_isAnagramEx3('TUFFAI', 'ABACA', 2, strict=False)
# assert (False, 5) == cy_isAnagramEx3('TUFFAI', 'ABACA', 2, strict=True)
# assert (True, 1) == cy_isAnagramEx3('AOPST', 'AOPSTU', 1, strict=False)
# assert (True, 1) == cy_isAnagramEx3('AOPST', 'AOPSTU', 1, strict=True)

# srcWord = 'straccio'
# dstWord = 'crosta'

# print('cy_isAnagramEx3("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
# print(cy_isAnagramEx3(srcWord, dstWord, maxDiffLength=maxDiffLength))

# t0 = time.time()
# for i in range(0):
#   cy_isAnagramEx3('', '')
#   not cy_isAnagramEx3('corto', 'trOca')
#   not cy_isAnagramEx3('c', '')
#   cy_isAnagramEx3('cortao', 'trocao')
#   cy_isAnagramEx3('cortao', 'trocaox', 1)
#   not cy_isAnagramEx3('strappo', 'posta', 1)
#   cy_isAnagramEx3('strappo', 'posta', 2)
#   cy_isAnagramEx3('strappo', 'posta', 3)
#   not cy_isAnagramEx3('strappo', 'posta', 3, strict=True)
#   cy_isAnagramEx3('strappo', 'posta', 2, strict=True)
#   cy_isAnagramEx3('LEVI', 'ABILE', 2, )
#   not cy_isAnagramEx3('TUFFAI', 'ABACA', 2, )
# print(f'{(time.time() - t0):.4f}s')


In [ ]:

# t0 = time.time()
# for i in range(0):
#   cy_isAnagramEx3('', '')
#   not cy_isAnagramEx3('corto', 'trOca')
#   not cy_isAnagramEx3('c', '')
#   cy_isAnagramEx3('cortao', 'trocao')
#   cy_isAnagramEx3('cortao', 'trocaox', 1)
#   not cy_isAnagramEx3('strappo', 'posta', 1)
#   cy_isAnagramEx3('strappo', 'posta', 2)
#   cy_isAnagramEx3('strappo', 'posta', 3)
#   not cy_isAnagramEx3('strappo', 'posta', 3, strict=True)
#   cy_isAnagramEx3('strappo', 'posta', 2, strict=True)
#   cy_isAnagramEx3('LEVI', 'ABILE', 2, )
#   not cy_isAnagramEx3('TUFFAI', 'ABACA', 2, )
# print(f'{(time.time() - t0):.4f}s')

# isAnagramEx2

In [ ]:
import time

def _isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=0, strict=False, failFast=False):
  if failFast and abs(sourceLength - targetLength) > maxDiffLength:
    return False, -1

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:
  #   wordDiffLength = abs(sourceLength - targetLength)
  #   if wordDiffLength > maxDiffLength:
  #       return False

  i = 0
  j = 0
  diff = 0
  same = 0
  if sourceLength != 0 and targetLength != 0:
    while i < sourceLength and j < targetLength:
      sourceCh = sortedSourceChars[i]
      targetCh = sortedTargetChars[j]
      if sourceCh == targetCh:
        i += 1
        j += 1
        same += 1
      else:
        if sourceCh < targetCh:
          i += 1
        else:
          j += 1

  diff = max(sourceLength, targetLength) - same
  #print(sortedSourceChars, sortedTargetChars, diff, same)
  isLooseAnagram = diff <= maxDiffLength
  res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)
  return res, diff


def isAnagramEx2(sourceWord, targetWord, maxDiffLength=0, strict=False, failFast=False):
  sortedSourceChars = sorted(sourceWord)
  sortedTargetChars = sorted(targetWord)

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:


  sourceLength = len(sortedSourceChars)
  targetLength = len(sortedTargetChars)

  return _isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=maxDiffLength, strict=strict, failFast=failFast)

maxDiffLength = 2

assert (True, 0) == isAnagramEx2('', '')
assert (False, 2) == isAnagramEx2('corto', 'trOca')
assert (False, 4) == isAnagramEx2('corto', 'trOcaul')
assert (False, 1) == isAnagramEx2('c', '')
assert (True, 1) == isAnagramEx2('cortao', 'trocaox', 1)
assert (True, 0) == isAnagramEx2('cortao', 'trocao')
assert (False, 2) == isAnagramEx2('strappo', 'posta', 1)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 2)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 3)
assert (False, 2) == isAnagramEx2('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagramEx2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagramEx2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagramEx2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagramEx2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagramEx2('AOPST', 'AOPSTU', 1, strict=True)

srcWord = 'straccio'
dstWord = 'crosta'

print('isAnagramEx2("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx2(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(0):
  isAnagramEx2('', '')
  not isAnagramEx2('corto', 'trOca')
  not isAnagramEx2('c', '')
  isAnagramEx2('cortao', 'trocao')
  isAnagramEx2('cortao', 'trocaox', 1)
  not isAnagramEx2('strappo', 'posta', 1)
  isAnagramEx2('strappo', 'posta', 2)
  isAnagramEx2('strappo', 'posta', 3)
  not isAnagramEx2('strappo', 'posta', 3, strict=True)
  isAnagramEx2('strappo', 'posta', 2, strict=True)
  isAnagramEx2('LEVI', 'ABILE', 2, )
  not isAnagramEx2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


In [ ]:
import time
import math
from collections import Counter

sum = __builtins__.sum

def isAnagram_Counter(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sourceCounter = Counter(sourceWord)
  targetCounter = Counter(targetWord)

  # swap? (lengthier first!)
  if len(sourceWord) < len(targetWord):
    sourceCounter, targetCounter = targetCounter, sourceCounter

  diffCounter = sourceCounter - targetCounter

  diffSum = sum(diffCounter.values())

  isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
  return isAnagram, diffSum

print("->", isAnagram_Counter('corto', 'trOca'))


assert (True, 0) == isAnagram_Counter('', '')
assert (False, 2) == isAnagram_Counter('corto', 'trOca')
assert (False, 4) == isAnagram_Counter('corto', 'trOcaul')
assert (False, 1) == isAnagram_Counter('c', '')
assert (True, 1) == isAnagram_Counter('cortao', 'trocaox', 1)
assert (True, 0) == isAnagram_Counter('cortao', 'trocao')
assert (False, 2) == isAnagram_Counter('strappo', 'posta', 1)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 2)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 3)
assert (False, 2) == isAnagram_Counter('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagram_Counter('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagram_Counter('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagram_Counter('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagram_Counter('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagram_Counter('AOPST', 'AOPSTU', 1, strict=True)
assert (True, 1) == isAnagram_Counter('BAVA', 'ABAVI', 1, strict=True)

t0 = time.time()
for i in range(0):
  isAnagram_Counter('', '')
  not isAnagram_Counter('corto', 'trOca')
  not isAnagram_Counter('c', '')
  isAnagram_Counter('cortao', 'trocao')
  isAnagram_Counter('cortao', 'trocaox', 1)
  not isAnagram_Counter('strappo', 'posta', 1)
  isAnagram_Counter('strappo', 'posta', 2)
  isAnagram_Counter('strappo', 'posta', 3)
  not isAnagram_Counter('strappo', 'posta', 3, strict=True)
  isAnagram_Counter('strappo', 'posta', 2, strict=True)
  isAnagram_Counter('LEVI', 'ABILE', 2, )
  not isAnagram_Counter('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


In [ ]:
import time
from copy import copy

class Counter2(dict):

  def __init__(self, word):
    self.word = word
    for ch in word:
      if ch not in self:
        self[ch] = 0
      self[ch] += 1
      # if ch not in self:
      #   self[ch] = word.count(ch)

  def __add__(self, otherCounter):
    newCounter = copy(self)
    for k, v in otherCounter.items():
      if k not in newCounter:
        newCounter[k] = 0
      newCounter[k] += v

    return newCounter

  def __sub__(self, otherCounter):
    newCounter = copy(self)
    for k, v in otherCounter.items():
      if k not in newCounter:
        newCounter[k] = 0
      else:
        newCounter[k] -= v
        if newCounter[k] <= 0:
          del newCounter[k]

    return newCounter

  def isAnagram(self, otherCounter, maxDiffLength=1, strict=False, failFast=False):
    sourceCounter, targetCounter = self, otherCounter
    sourceLength, targetLength = len(sourceCounter.word), len(targetCounter.word)
    if failFast and abs(targetLength - sourceLength) > maxDiffLength:
      return False, -1
    if sourceLength < targetLength:
      sourceCounter, targetCounter = targetCounter, sourceCounter
      sourceLength, targetLength = targetLength, sourceLength
    diffCounter = sourceCounter - targetCounter
    diffSum = sum(diffCounter.values())

    isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
    return isAnagram, diffSum


def isAnagram_Counter2(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sourceCounter = Counter2(sourceWord)
  targetCounter = Counter2(targetWord)

  # swap? (lengthier first!)
  if len(sourceWord) < len(targetWord):
    sourceCounter, targetCounter = targetCounter, sourceCounter

  diffCounter = sourceCounter - targetCounter

  diffSum = sum(diffCounter.values())

  isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
  return isAnagram, diffSum

print("->", isAnagram_Counter2('corto', 'trOca'))

word1 = Counter2('corto')
word2 = Counter2('trOca')
display(word1)
display(word2)
display(word1 + word2)
display(word1 - word2)
display(Counter('corto') - Counter('trOca'))

assert (True, 0) == isAnagram_Counter2('', '')
assert (False, 2) == isAnagram_Counter2('corto', 'trOca')
assert (False, 4) == isAnagram_Counter2('corto', 'trOcaul')
assert (False, 1) == isAnagram_Counter2('c', '')
assert (True, 1) == isAnagram_Counter2('cortao', 'trocaox', 1)
assert (True, 0) == isAnagram_Counter2('cortao', 'trocao')
assert (False, 2) == isAnagram_Counter2('strappo', 'posta', 1)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 2)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 3)
assert (False, 2) == isAnagram_Counter2('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagram_Counter2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagram_Counter2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagram_Counter2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagram_Counter2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagram_Counter2('AOPST', 'AOPSTU', 1, strict=True)
assert (True, 1) == isAnagram_Counter2('BAVA', 'ABAVI', 1, strict=True)
assert (True, 1) == Counter2('BAVA').isAnagram(Counter2('ABAVI'), maxDiffLength=1)

t0 = time.time()
for i in range(0):
  isAnagram_Counter2('', '')
  not isAnagram_Counter2('corto', 'trOca')
  not isAnagram_Counter2('c', '')
  isAnagram_Counter2('cortao', 'trocao')
  isAnagram_Counter2('cortao', 'trocaox', 1)
  not isAnagram_Counter2('strappo', 'posta', 1)
  isAnagram_Counter2('strappo', 'posta', 2)
  isAnagram_Counter2('strappo', 'posta', 3)
  not isAnagram_Counter2('strappo', 'posta', 3, strict=True)
  isAnagram_Counter2('strappo', 'posta', 2, strict=True)
  isAnagram_Counter2('LEVI', 'ABILE', 2, )
  not isAnagram_Counter2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


# TreeNode

In [ ]:
import random

def counterCompare(counterA, counterB):
  # counterA = Counter2(wordA)
  # counterB = Counter2(wordB)
  #isAnagram, diff = counterA.isAnagram(counterB, maxDiffLength=1, strict=True, failFast=True)
  isAnagram, diff = isAnagramEx2(counterA.word, counterB.word, maxDiffLength=1, strict=True)
  #print(counterA.word, counterB.word, (isAnagram, diff))
  return -1 if isAnagram else 1

def wordCompare(wordA, wordB):
  isAnagram, diff = isAnagramEx2(wordA, wordB, maxDiffLength=1, strict=True, failFast=True)
  #print(counterA.word, counterB.word, (isAnagram, diff))
  return -1 if isAnagram else 1

class TreeNode:
  def __init__(self, word):
    self.left = None
    self.right = None
    self.data = word

  def insert(self, word, compare=wordCompare):
    data = word

    y = None
    node = self
    i = 0
    while node is not None:
      i += 1
      if node.data:
        y = node
        comparison = compare(node.data, data)
        if comparison < 0:
          node = node.left
        else:
          node = node.right

    # if random.choice([0,1,2,4,5]) == 0:
    #   print(i)

    if y is None:
      self.data = data
    elif comparison < 0:
      y.left = TreeNode(word)
    else:
      y.right = TreeNode(word)

  def search(self, word, compare=wordCompare):
    results = []

    data = word

    y = None
    node = self
    while node is not None:
      if node.data:
        y = node
        comparison = compare(node.data, data)
        #print(comparison, node.data.word, data.word)
        if comparison < 0:
          results.append(node.data)
          node = node.left
        else:
          node = node.right

    return results


def printTree(node, level=0):
  if node != None:
    printTree(node.left, level + 1)
    print(' ' * 4 * level + '-> ' + str(node.word))
    printTree(node.right, level + 1)



t0 = time.time()
root = TreeNode('@')
shuffledWords = filteredWords[:]
random.shuffle(shuffledWords)
for i, word in enumerate(shuffledWords[:0]):
  root.insert(word)
  if i % 1000 == 0:
    print(i, word)
print(f'build took {(time.time() - t0):.2f}')

#printTree(root)

t0 = time.time()
word = shuffledWords[1]
res = root.search(word)
print(f'search {word} took {(time.time() - t0):.2f}')

display(res)


In [ ]:
#print(shuffledWords[:100])
t0 = time.time()
word = random.choice(shuffledWords[:5000])
res = root.search(word)
print(f'search {word} took {(time.time() - t0):.2f}')

display(res)


# Trie

In [ ]:
# http://stevehanov.ca/blog/index.php?id=114
# By Steve Hanov, 2011. Released to the public domain
import time
import sys

DICTIONARY = "/usr/share/dict/words";
TARGET = "SCORTA"
MAX_COST = 4

# Keep some interesting statistics
NodeCount = 0
WordCount = 0

# The Trie data structure keeps a set of words, organized with one node for
# each letter. Each node has a branch for each letter that may follow it in the
# set of words.
class TrieNode:
    def __init__(self):
        self.word = None
        self.children = {}

        global NodeCount
        NodeCount += 1

    def insert( self, word ):
        node = self
        for letter in word:
            if letter not in node.children:
                node.children[letter] = TrieNode()

            node = node.children[letter]

        node.word = word

# read dictionary file into a trie
start = time.time()

trie = TrieNode()
for word in filteredWords:
    WordCount += 1
    trie.insert( word )

end = time.time()
print("Build took %g s" % (end - start))

print("Read %d words into %d nodes" % (WordCount, NodeCount))

# The search function returns a list of all words that are less than the given
# maximum distance from the target word
def search( word, maxCost ):

    # build first row
    currentRow = range( len(word) + 1 )

    results = []

    # recursively search each branch of the trie
    for letter in trie.children:
        searchRecursive( trie.children[letter], letter, word, currentRow,
            results, maxCost )

    return results

# This recursive helper is used by the search function above. It assumes that
# the previousRow has been filled in already.
def searchRecursive( node, letter, word, previousRow, results, maxCost ):

    columns = len( word ) + 1
    currentRow = [ previousRow[0] + 1 ]

    # Build one row for the letter, with a column for each letter in the target
    # word, plus one for the empty string at column 0
    for column in range( 1, columns ):

        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1

        if word[column - 1] != letter:
            replaceCost = previousRow[ column - 1 ] + 1
        else:
            replaceCost = previousRow[ column - 1 ]

        currentRow.append( min( insertCost, deleteCost, replaceCost ) )

    # if the last entry in the row indicates the optimal cost is less than the
    # maximum cost, and there is a word in this trie node, then add it.
    if currentRow[-1] <= maxCost and node.word != None:
        results.append( (node.word, currentRow[-1] ) )

    # if any entries in the row are less than the maximum cost, then
    # recursively search each branch of the trie
    if min( currentRow ) <= maxCost:
        for letter in node.children:
            searchRecursive( node.children[letter], letter, word, currentRow,
                results, maxCost )

start = time.time()
results = search( TARGET, MAX_COST )
end = time.time()

#for result in results: print(result)

print("Search took %g s" % (end - start))



In [ ]:
import random

startWord = random.choice(filteredWords)
print("START WORD:", startWord)

start = time.time()
results = search(startWord, 3)
end = time.time()

print("Search took %g s" % (end - start))

validEndWord = random.choice(results)

print("END WORD:", validEndWord)


# findAnagramsEx

In [ ]:
def findAnagramsEx(wordList, word, maxDiffLength=1, strict=False):
    results = []
    wordLength = len(word)
    for entry in wordList:
        entryLength = len(entry)
        if word == entry or entryLength < wordLength - maxDiffLength or entryLength > wordLength + maxDiffLength:
            continue
        isAnagram, diff = isAnagramEx2(entry, word, maxDiffLength=maxDiffLength, strict=strict)
        if isAnagram:
            results.append((entry, diff))

    return results


# pd_findAnagrams

In [ ]:
from IPython.core.display import display_html

def pd_findAnagrams(df, word, maxDiffLength=1, strict=False):
    results = []

    t0 = time.time()

    if word not in df.index:
        return results

    row = df.loc[word]

    sub_df = df.iloc[:]
    sum_ds = sub_df.sum(axis=1)
    # display(sum_ds)
    isInRange = abs(sum_ds - len(word)) <= maxDiffLength
    sub_df = sub_df[isInRange]
    # display_html(f'<br>candidates:', raw=True)
    # display(sub_df)

    len_ds = sub_df.sum(axis=1)

    sub_df = sub_df - row

    sum_df = sub_df
    pos_sum_ds = sub_df[sub_df > 0].sum(axis=1)
    neg_sum_ds = sub_df[sub_df < 0].sum(axis=1)
    diff_ds = pd.DataFrame({'POS':pos_sum_ds, 'NEG': abs(neg_sum_ds)}).max(axis=1)
    # display(diff_ds)
    # display(pos_sum_ds, neg_sum_ds)
    sum_df = pd.concat([sum_df, pos_sum_ds.rename('POS'), neg_sum_ds.rename('NEG'), len_ds.rename('LEN'), diff_ds.rename('DIFF')], axis=1)

    # display(sum_df.loc['RASAVI'])
    # display(sum_df)

    betterIsAnagramInRange = (diff_ds <= maxDiffLength)

    anagrams = []
    diffs = []
    if not strict:
        # display_html(f'<br>Anagrams of word: <b>{word} ({len(word)}) </b> maxDiffLength: <b>{maxDiffLength}</b>', raw=True)

        anagrams_df = sum_df[betterIsAnagramInRange]
        # display(anagrams_df)

        anagrams = anagrams_df.index.to_list()
        diffs = anagrams_df['DIFF'].to_list()
    else:
        # display_html(f'<br>Anagrams of word: <b>{word} ({len(word)}) </b> maxDiffLength: <b>{maxDiffLength} (strict)</b>', raw=True)

        strict_anagrams_df = sum_df[(betterIsAnagramInRange) & (abs(sum_df['LEN'] - len(word)) == maxDiffLength)]
        # display(strict_anagrams_df)

        anagrams = strict_anagrams_df.index.to_list()
        diffs = strict_anagrams_df['DIFF'].to_list()

    # display_html(f'<br>Anagrams of word: <b>{word} ({len(word)}) </b> maxDiffLength: <b>0</b>', raw=True)
    # display(sum_df[(betterIsAnagramInRange) & (sum_df['LEN'] == len(word))])

    # print(anagrams)
    try:
      word_idx = anagrams.index(word)
      anagrams.pop(word_idx)
      diffs.pop(word_idx)
    except ValueError: # not in list
        pass

    results = list(zip(anagrams, diffs))

    print(f"elapsed {(time.time() - t0):.2f}s")

    return results

pd_findAnagrams(df, "ALLIEVA", maxDiffLength=1, strict=False)
res = pd_findAnagrams(df, "ALLIEVA", maxDiffLength=1, strict=False)
anagrams_7 = [t[0] for t in res if t[1] == 0 and len(t[0]) == 7]
print(anagrams_7)
anagrams_6 = [t[0] for t in res if len(t[0]) == 6]
display_html(f'<br>Trovati {len(anagrams_6)} anagrammi parziali di <b>ALLIEVA</b> di 6 lettere:', raw=True)
display(anagrams_6)

In [ ]:
import random

newStartWord = random.choice(filteredWords)
print("START WORD:", startWord)

anagramsDict = {0:[], 1:[], 2:[]}
randomFilteredWords = filteredWords[:]
random.shuffle(filteredWords[:])
randomFilteredWords = sorted(randomFilteredWords)[:100]

anagrams = findAnagramsEx(randomFilteredWords, newStartWord, maxDiffLength=1, strict=False)
for anagram in anagrams:
    anagramsDict[anagram[1]].append(anagram[0])

print(anagrams)
#print(anagramsDict[0][:10])

In [ ]:
print(anagramsDict[1])

# generateChain

In [ ]:
import time

def generateChain(quiet=False, startWord=None, strict=False):
    startWords = wordsOfLength(START_WORD_LENGTHS)

    restart = True

    while restart:
        restart = False
        startWord = random.choice(startWords)
        print("START WORD:", startWord)

        steps = [startWord]

        prevStep = (startWord, 0)
        #self.info.text = ""
        for i in range(6):
            choice = random.choice(["anagram", "levenshtein"])
            if choice == "anagram":
                if not quiet: print("Finding anagrams...", end="")
                startTime = time.time()
                # anagrams = findAnagramsEx(filteredWords, prevStep[0], maxDiffLength=1, strict=strict)
                anagrams = pd_findAnagrams(df, prevStep[0], maxDiffLength=1, strict=strict)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(anagrams) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(anagrams))
                    break
            else:
                if not quiet: print("Finding levenshtein...", end="")
                startTime = time.time()
                results = search(prevStep[0], 1)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(results) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(results))
                    break

            done = False
            while not done:
                if choice == "anagram":
                    nextStep = random.choice(anagrams)
                else:
                    nextStep = random.choice(results)

                if nextStep[0] not in steps:
                    steps.append(nextStep[0])
                    done = True

            resultsCount = len(results) if choice == "levenshtein" else len(anagrams)
            if not quiet: print(nextStep, resultsCount, choice)
            prevStep = nextStep

        print("restart", restart)
        print()

    endWord = steps[-1]
    print(steps[1:-1])
    print("END WORD:", endWord)

    return steps

t0 = time.time()
steps = generateChain(startWord=None, strict=True)
print("Elapsed {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

# Levenshtein

In [ ]:
!pip install rapidfuzz
import rapidfuzz

levenshtein = rapidfuzz.distance.Levenshtein.distance


In [ ]:
for w,s in zip(['CORTO', 'ORTO', 'SCORTA'], ['CORTI', 'TORTO', 'SCARTO']):
  print(f'{w} {s} {levenshtein(w,s)}')


# checkValidChain

In [ ]:
def isInDictionary(word, wordList):
    return word in wordList

class WordStatus():
  UNCHECKED = 0
  VALID = 1
  INVALID = 2
  NOT_IN_DICT = 3

def checkValidChain(startWord, endWord, wordChain):
    prevWord = startWord
    isOk = True
    statuses = []
    print()
    for i, word in enumerate(wordChain + [endWord]):
        inDictionary = isInDictionary(word, validationDatabase)
        # results = [t[0] for t in search(prevWord, 1)]
        # anagrams = [t[0] for t in findAnagramsEx(filteredWords, prevWord, maxDiffLength=1, strict=False)]
        # anagrams = [t[0] for t in pd_findAnagrams(df, prevWord, maxDiffLength=1, strict=False)]
        # isAnagram = word in [t[0] for t in pd_findAnagrams(df, prevWord, maxDiffLength=1, strict=False)]
        isAnagram, diff = isAnagramEx2(word, prevWord, maxDiffLength=1, strict=False)
        isLevenshtein = levenshtein(word, prevWord) <= 1
        sameAsStartOrEndWord = (i < len(wordChain)) and word in [startWord, endWord]
        isDuplicate = False
        try:
            idx = wordChain.index(word)
            isDuplicate = idx != i
        except ValueError: # not in list
            pass
        isWordOk = inDictionary and (isLevenshtein or isAnagram) and not sameAsStartOrEndWord and not isDuplicate
        # print('inDict:', inDictionary, ', levenshtein:', word in results, ', anagrams:',
        #       word in anagrams, ', samemeStartOrEndOrd:', sameAsStartOrEndWord, ', isDuplicate:', isDuplicate)
        if not inDictionary:
            print("✖️", word)
            statuses.append(WordStatus.NOT_IN_DICT)
        else:
            print("✔️" if isWordOk else "❌", word)
            statuses.append(WordStatus.VALID if isWordOk else WordStatus.INVALID)

        if not isWordOk:
            isOk = False
        prevWord = word

    return isOk, statuses

startWord, endWord = "TRITA", "TRIAL"

print("START WORD:", startWord)
print("END WORD  :", endWord)

t0 = time.time()
checkValidChain(startWord, endWord, ["LITRA", "TRIAL"])
print("Elapsed {0:.2f}s".format(time.time() - t0))


checkValidChain

In [ ]:
def displayCheckValidChain(startWord, endWord, wordChain):
    print()
    print(startWord)
    ok, statuses = checkValidChain(startWord, endWord, wordChain)
    print(endWord)

    print()
    print("👍 WON" if ok else "👎 LOST")

    print()


In [ ]:
# regenerate start/end words
t0 = time.time()
steps = generateChain(quiet=True, startWord='piccolo')
print("\nGenerate {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

print("START WORD:", startWord)
print("END WORD  :", endWord)

In [ ]:
wordChain = [
    'pantera',
    'pentola',
    'pantofola',
    'osteria',
    'ristorai',
    'piantala',
]
wordChain = [w.strip().upper() for w in wordChain]
#print("GUESS WORD CHAIN", wordChain)

t0 = time.time()
displayCheckValidChain(startWord, endWord, wordChain)
print("Validate {0:.2f}s".format(time.time() - t0))

In [ ]:
# STEPS

print("COMPUTED WORD CHAIN", steps)

In [ ]:
# find in dictionary

findWord = 'muovi'

search(findWord.strip().upper(), 0)


In [ ]:
print([hex(ord(ch)) for ch in "✖️"])
print([hex(ord(ch)) for ch in "✔️"])
print([hex(ord(ch)) for ch in "❌"])
"👍"
hex(ord("👎"))

# anagrams dict

In [ ]:
t0 = time.time()

anagramsDict = {}
wordsByLength = {}
wordsBySortedAndLength = {}
for w in filteredWords[:]:
  sortedWord = ''.join(sorted(w))
  if sortedWord not in anagramsDict:
    anagramsDict[sortedWord] = set()
  anagramsDict[sortedWord].add(w)
  # moreAnagrams = findAnagramsEx(filteredWords, w, maxDiffLength=1, strict=True)
  # for item in moreAnagrams:
  #   anagramsDict[sortedWord].add(item[0])
  length = len(w)
  if length not in wordsByLength:
    wordsByLength[length] = set()
  wordsByLength[length].add(w)
  if w not in wordsBySortedAndLength:
    wordsBySortedAndLength[w] = [None, None, None]
  wordsBySortedAndLength[w][0] = sortedWord
  wordsBySortedAndLength[w][1] = length
  counter = Counter2(w)
  wordsBySortedAndLength[w][2] = counter


print(f'{(time.time() - t0):.2f}s')

display([item for item in random.choices(list(anagramsDict.items()), k=10)])
display([(k, list(v)[:5]) for k, v in wordsByLength.items()])
display([item for item in random.choices(list(wordsBySortedAndLength.items()), k=10)])


In [ ]:
for k in sorted(wordsByLength.keys()):
  print(f'{k}:', len(wordsByLength[k]))

In [ ]:

# t0 = time.time()

# wordList = filteredWords[:21]
# for i, w in enumerate(wordList):
#   if i % 100 == 0:
#     print(f'{i:6}/{len(wordList)} {w}')
#   sortedWord, wordLength, sourceCounter = wordsBySortedAndLength[w]

#   candidates = []
#   targetLength = wordLength - 1
#   if targetLength in wordsByLength:
#     candidates = list(wordsByLength[targetLength])
#   targetLength = wordLength + 1
#   if targetLength in wordsByLength:
#     candidates += list(wordsByLength[targetLength])

#   for targetWord in candidates:
#     sortedTargetWord, targetLength, targetCounter = wordsBySortedAndLength[targetWord]
#     isAnagram, diff = _isAnagramEx2(sortedWord, sortedTargetWord, wordLength, targetLength, maxDiffLength=1, strict=True)

#     # if len(w) < len(targetWord):
#     #   sourceCounter, targetCounter = targetCounter, sourceCounter
#     # diffCounter = sourceCounter - targetCounter
#     # diffSum = sum(diffCounter.values())
#     # isAnagram = diffSum == 1

#     #sourceCounter.isAnagram(targetCounter, maxDiffLength = 1)

#     if isAnagram:
#       # print("->", targetWord)
#       anagramsDict[sortedWord].add(targetWord)
#       # display(anagramsDict[sortedWord])

# print(f'{(time.time() - t0):.2f}s')


In [ ]:
word = 'ABAVI'
sortedWord, wordLength, sourceCounter = wordsBySortedAndLength[word]
display(anagramsDict[sortedWord])

_isAnagramEx2(list('BAVA'), list('ABAVI'), 4, 5, maxDiffLength=1, strict=True)


In [ ]:
print('LUCRAMELO'.__hash__())
print('LUCRAMELO'.__hash__())
print('lucramelo'.__hash__())

for k,v in wordsByLength.items():
  print(k, len(v))

#findAnagramsEx(filteredWords, 'TAVOLIERE', maxDiffLength=1, strict=True)

# server connect

In [ ]:
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_2BWTWXDY2V4ON7JLQ4HILKDO-QMTVJN4LDJALI4EQ")



# server functions

In [ ]:
@anvil.server.callable
def srvGenerateWordChain(quiet=True, startWord=None, strict=True):
  steps = generateChain(quiet=quiet, startWord=startWord, strict=strict)
  return steps

@anvil.server.callable
def srvValidateChain(startWord, endWord, wordChain):
  isOk, statuses = checkValidChain(startWord, endWord, wordChain)
  return isOk, statuses



# server wait

In [ ]:
anvil.server.wait_forever()

START WORD: SUBITO
elapsed 0.17s
elapsed 0.09s
elapsed 0.05s
levenshtein 9
restart True

START WORD: ARARE
elapsed 0.08s
elapsed 0.15s
elapsed 0.25s
elapsed 0.39s
elapsed 0.30s
restart False

['ARATE', 'REMATA', 'TARMATE', 'RAMATATE', 'MAHARATTE']
END WORD: AMATTERA
START WORD: AVVITI
elapsed 0.16s
anagram 6
restart True

START WORD: LAMIA
elapsed 0.10s
elapsed 0.09s
elapsed 0.17s
restart False

['MALARI', 'MALORI', 'MAORI', 'ROMBAI', 'FIBROMA']
END WORD: FIBROMI
START WORD: MUNDIO
elapsed 0.16s
levenshtein 9
restart True

START WORD: MINUS
elapsed 0.10s
levenshtein 2
restart True

START WORD: SAGLIO
elapsed 0.16s
elapsed 0.26s
levenshtein 9
restart True

START WORD: METILE
elapsed 0.15s
elapsed 0.27s
elapsed 0.40s
levenshtein 5
restart True

START WORD: SCOSSE
elapsed 0.15s
levenshtein 7
restart True

START WORD: CONGHE
elapsed 0.16s
anagram 2
restart True

START WORD: ROGITO
levenshtein 6
restart True

START WORD: COMPII
elapsed 0.16s
levenshtein 9
restart True

START WORD: DECADO
el

KeyboardInterrupt: ignored

In [ ]:
anvil.server.disconnect()

Anvil websocket closed (code 1000, reason=b'')
